## 使用KNN分类器(k-d树版本)判断乳腺肿瘤是否为良性

In [1]:
import numpy as np

In [7]:
# 加载数据集
X = np.genfromtxt('wpbc.data', delimiter=',', usecols=range(2, 32))
X

array([[3.100e+01, 1.802e+01, 2.760e+01, ..., 3.140e-01, 1.170e-01,
        2.677e-01],
       [6.100e+01, 1.799e+01, 1.038e+01, ..., 7.119e-01, 2.654e-01,
        4.601e-01],
       [1.160e+02, 2.137e+01, 1.744e+01, ..., 3.414e-01, 2.032e-01,
        4.334e-01],
       ...,
       [1.200e+01, 1.717e+01, 2.919e+01, ..., 2.141e-01, 9.520e-02,
        3.362e-01],
       [3.000e+00, 2.142e+01, 2.284e+01, ..., 5.411e-01, 2.215e-01,
        2.832e-01],
       [6.000e+00, 1.670e+01, 2.813e+01, ..., 3.455e-01, 1.317e-01,
        3.035e-01]])

In [6]:
y = np.genfromtxt('wpbc.data', delimiter=',', usecols=1, dtype=np.str)
y

array(['N', 'N', 'N', 'N', 'R', 'R', 'N', 'R', 'N', 'N', 'N', 'N', 'N',
       'R', 'N', 'R', 'N', 'R', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'R', 'N', 'N', 'R', 'N',
       'R', 'R', 'N', 'R', 'R', 'N', 'N', 'R', 'N', 'R', 'N', 'R', 'N',
       'N', 'N', 'N', 'N', 'R', 'N', 'N', 'R', 'N', 'N', 'R', 'N', 'N',
       'N', 'N', 'R', 'N', 'R', 'N', 'N', 'N', 'N', 'N', 'R', 'N', 'N',
       'N', 'N', 'N', 'R', 'N', 'N', 'N', 'N', 'N', 'N', 'R', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'R', 'N', 'N', 'N', 'R', 'R', 'N', 'R', 'N', 'R', 'N', 'N',
       'N', 'N', 'N', 'R', 'N', 'N', 'N', 'N', 'R', 'N', 'N', 'R', 'R',
       'N', 'R', 'R', 'N', 'N', 'R', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'R', 'N', 'R', 'N', 'R', 'N', 'N', 'R', 'N', 'R', 'N', 'N', 'R',
       'N', 'R', 'N', 'R', 'N', 'R', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'R', 'R', 'N', 'N', 'N

In [5]:
y = np.where(y == 'B', 1, 0)
y

array(0)